# **STEP ONE**

In [ ]:
!pip install -q mediapipe==0.10.0

# **STEP TWO**

In [ ]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2

import numpy as np
import cv2 as cv

from IPython.display import display, Javascript, Image, clear_output, HTML
from google.colab.output import eval_js
import html
from base64 import b64decode, b64encode
import PIL
import io

import matplotlib.pyplot as plt
import time
import os
import glob
import csv

In [ ]:
# Class that is able to conviently store all poselandmarker data
# Input a poselandmarker dstruct and it outputs the x y or z of any specific body part
# lm_dstruct.nose.x
class lm_dstruct:
  def __init__(self, pll):
    self.nose = pll[0]
    self.left_eye_in = pll[1]
    self.left_eye = pll[2]
    self.left_eye_out = pll[3]
    self.right_eye_in = pll[4]
    self.right_eye = pll[5]
    self.right_eye = pll[6]
    self.left_ear = pll[7]
    self.right_ear = pll[8]
    self.left_mouth = pll[9]
    self.right_mouth = pll[10]
    self.left_shoulder = pll[11]
    self.right_shoulder = pll[12]
    self.left_elbow = pll[13]
    self.right_elbow = pll[14]
    self.left_wrist = pll[15]
    self.right_wrist = pll[16]
    self.left_pinky = pll[17]
    self.right_pinky = pll[18]
    self.left_index = pll[19]
    self.right_index = pll[20]
    self.left_thumb = pll[21]
    self.right_thumb = pll[22]
    self.left_hip = pll[23]
    self.right_hip = pll[24]
    self.left_knee = pll[25]
    self.right_knee = pll[26]
    self.left_ankle = pll[27]
    self.right_ankle = pll[28]
    self.left_heel = pll[29]
    self.right_heel = pll[30]
    self.left_foot_index = pll[31]
    self.right_foot_index = pll[32]

  def angle(self, pointa, pointb, pointc):
    pointax = pointa.x - pointb.x
    pointay = pointa.y - pointb.y
    pointaz = pointa.z - pointb.z
    pointcx = pointc.x - pointb.x
    pointcy = pointc.y - pointb.y
    pointcz = pointc.z - pointb.z
    return np.arccos(np.dot((pointax, pointay, pointaz), (pointcx, pointcy, pointcz)) / (np.sqrt(pointax**2 + pointay**2 + pointaz**2) * np.sqrt(pointcx**2 + pointcy**2 + pointcz**2))) * 180 / np.pi

  def feedback(self, exercise):
    with open('{exercise}.csv') as anglefile:
      anglereader = csv.reader(anglefile)
      for row in anglereader:
        print(row)

Make sure to import **Pose Landmarker Lite** into Drive folder before next step

Can be found in Drive folder: https://drive.google.com/drive/folders/1RFTh9ed50zVZaIyutPCo6-VTEQanqE4_?usp=sharing

If needed, download latest from MediaPipe website: https://developers.google.com/mediapipe/solutions/vision/pose_landmarker/index#models

# **STEP THREE**

In [ ]:
model_path = '/content/drive/MyDrive/Exercise App Resources/pose_landmarker_lite.task'
BaseOptions = python.BaseOptions # Basic Options Object
PoseLandmarker = vision.PoseLandmarker # Pose Landmarker Model Object
PoseLandmarkerOptions = vision.PoseLandmarkerOptions # Overall Options Object
VisionRunningMode = vision.RunningMode # Mode Object (Image, Video, Livestream)
PoseLandmarkerResult = vision.PoseLandmarkerResult # Stores result from model
out_video = cv.VideoWriter('/content/drive/MyDrive/Exercise App Resources/Videos/' + time.asctime(time.localtime()) + '.mp4',cv.VideoWriter_fourcc(*'DIVX'), 24, (640, 480))
landmarks_list = []
feedback_list = [""]

# Create a pose landmarker instance with the live stream mode:
def print_result(result: PoseLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
  if result.pose_landmarks:
    annotated_image = np.copy(output_image.numpy_view())
    landmarks_list.append(result.pose_landmarks[0]) # Adds landmark data for pic to list
    for landmark in result.pose_landmarks[0]:
      pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
      pose_landmarks_proto.landmark.extend(
          [landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in result.pose_landmarks[0]])
      solutions.drawing_utils.draw_landmarks(
          annotated_image,
          pose_landmarks_proto,
          solutions.pose.POSE_CONNECTIONS,
          solutions.drawing_styles.get_default_pose_landmarks_style())

    # print("Frame", timestamp_ms)
    lol = lm_dstruct(landmarks_list[-1])
    feedback_temp = lol.feedback("squat")
    if feedback_list[-1] != feedback_temp:
      print(feedback_temp)
    feedback_list.append(feedback_temp)
    out_video.write(annotated_image)

options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result) # Callback function to deal w/ results


### CAMERA FUNCTIONALITY ###

def jsob_to_image(js_object):
  # decode base64 image
  image_bytes = b64decode(js_object.split(',')[1])
  # convert bytes to numpy array
  img_array = np.frombuffer(image_bytes, dtype=np.uint8)
  # convert numpy array into OpenCV BGR
  frame = cv.imdecode(img_array, flags=1)

  return frame

def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }
      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);


      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);
      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="blue: red; font-weight: bold;">' +
          'click here to stop the video</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();
      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640;
      captureCanvas.height = 480;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame() {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }
      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();



      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame():
  data = eval_js('stream_frame()')
  return data

### END OF CAMERA CODE ###

with PoseLandmarker.create_from_options(options) as landmarker:
  frame_timestamp_ms = 0
  video_stream()
  while True:
    frame_timestamp_ms += 1
    frame_js = video_frame()
    if not frame_js:
      break
    img = jsob_to_image(frame_js["img"])
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img)

    landmarker.detect_async(mp_image, frame_timestamp_ms)

out_video.release()

<IPython.core.display.Javascript object>

Bad Form - Squat Not Deep Enough
Bad Form - Back Not Straight At Beginning
Bad Form - Squat Not Deep Enough
Bad Form - Back not straight in Squat
Bad Form - Back Not Straight At Beginning
Bad Form - Squat Not Deep Enough
Bad Form - Back Not Straight At Beginning
Bad Form - Squat Not Deep Enough
Bad Form - Back Not Straight At Beginning
Bad Form - Squat Not Deep Enough
Bad Form - Back Not Straight At Beginning
Bad Form - Squat Not Deep Enough
Bad Form - Back Not Straight At Beginning
Bad Form - Squat Not Deep Enough
Bad Form - Back Not Straight At Beginning
Bad Form - Squat Not Deep Enough
Bad Form - Back not straight in Squat
Good Form
Bad Form - Squat Not Deep Enough
Good Form
Bad Form - Back not straight in Squat
Bad Form - Squat Not Deep Enough
Good Form
Bad Form - Back not straight in Squat
Good Form
Bad Form - Squat Not Deep Enough
Bad Form - Back not straight in Squat
Bad Form - Squat Not Deep Enough
Bad Form - Back not straight in Squat
Good Form
Bad Form - Back not straight in 

In [ ]:
lol = lm_dstruct(landmarks_list[20])

In [ ]:
lol.angle(lol.left_shoulder, lol.left_elbow, lol.left_wrist)

117.35933841748007